In [1]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pickle
import re
from bs4 import BeautifulSoup
from sys import stdout

In [2]:
def save_to_pickle(file,name):
    with open(name + '.txt', 'wb') as f:
        pickle.dump(file, f)

# Marvel editors and writers

In [111]:
with open('marvel_comics_editor_writer.txt', 'rb') as f:
    marvel_ed_wr = pickle.load(f)

In [156]:
new_columns = ['URL', 'Gender', 'Birth', 'Death']
gender_ed_wr = pd.DataFrame(columns=new_columns)

for URL in marvel_ed_wr['Writer URL'].unique():
    if (URL is not '') and ('/wiki/' in URL):
        if URL[0] is not '/':
            URL = URL[1:]
            
        # Initialize variables
        gender, birth, death = '', '', ''

        # Get URL and use html parser
        URL = 'https://marvel.fandom.com' + URL

        r = requests.get(URL)
        page_body = r.text

        soup = BeautifulSoup(page_body, 'html.parser')

        # If URL links to the Editor/Writer main page
        if soup.find_all('span', style='font-weight:bold;'):
            span = soup.find_all('span', style='font-weight:bold;')
            if span:
                a = span[0].find_all('a', href=True)
                if a:
                    URL = 'https://marvel.fandom.com' + a[0]['href']
                    # html parser on the real page
                    r = requests.get(URL)
                    page_body = r.text

                    soup = BeautifulSoup(page_body, 'html.parser')

        # Special case
        if '/wiki/User:Ultimantium' in URL:
            gender = 'Male'

        URL = URL.replace('https://marvel.fandom.com','')

        # Parsing
        side_tab = soup.find_all('div', class_='infobox')

        # Gender
        for p in side_tab:
            for pp in p.find_all('div', style='clear:left;font-size:10px;text-align:center;border-top:1px solid #B5B7CF;'):
                for ppp in pp.find_all('div', style='width:90px;float:left;text-align:left;'):
                    if(ppp.text=="Gender"):
                        if pp.find('div', style='width:160px;float:left;text-align:right;'):
                            gender = pp.find('div', style='width:160px;float:left;text-align:right;').text
                            
        # Birth and death dates
        for pp in p.find_all('div', style="width:125px;float:left;clear:left;border-top:1px solid #B5B7CF;"):
            if pp:
                birth = pp.text
        for pp in p.find_all('div', style="width:125px;float:left;border-top:1px solid #B5B7CF;"):
            if pp:
                death = pp.text

        if URL not in gender_ed_wr['URL'].unique():
            editor = pd.DataFrame([[URL, gender, birth, death]], columns = new_columns)
            gender_ed_wr = gender_ed_wr.append(editor)

gender_ed_wr.head()

,URL,Gender,Birth,Death
0,/wiki/Joe_Caramagna,Male,,
0,/wiki/Stan_Lee_(Earth-1218),Male,"December 28, 1922","November 12, 2018"
0,/wiki/Larry_Lieber,Male,"October 26, 1931",
0,/wiki/Paul_Gustavson,Male,"August 16, 1916","April 29, 1977"
0,/wiki/Ray_Gill,Male,,


In [163]:
save_to_pickle(gender_ed_wr,'marvel_wr_attributes')

In [23]:
with open('marvel_wr_attributes.txt', 'rb') as f:
    marvel_wr = pickle.load(f)

In [24]:
with open('marvel_ed_attributes.txt', 'rb') as f:
    marvel_ed = pickle.load(f)

In [25]:
marvel_ed_wr = marvel_wr.append(marvel_ed)

In [26]:
save_to_pickle(marvel_ed_wr,'marvel_ed_wr_attributes')

In [27]:
with open('marvel_ed_wr_attributes.txt', 'rb') as f:
    marvel_ed_wr = pickle.load(f)

In [28]:
marvel_ed_wr

,URL,Gender,Birth,Death
0,/wiki/Joe_Caramagna,Male,,
0,/wiki/Stan_Lee_(Earth-1218),Male,"December 28, 1922","November 12, 2018"
0,/wiki/Larry_Lieber,Male,"October 26, 1931",
0,/wiki/Paul_Gustavson,Male,"August 16, 1916","April 29, 1977"
0,/wiki/Ray_Gill,Male,,
...,...,...,...,...
0,/wiki/C.B._Cebulski_(Earth-1218),Male,,
0,/wiki/John_Barber,Male,,
0,/wiki/User:Ultimantium,male,,
0,/wiki/Jeff_Youngquist,Male,,


# DC editors and writers

In [286]:
with open('dc_comics_editor_writer.txt', 'rb') as f:
    dc_ed_wr = pickle.load(f)

In [315]:
dc_ed_wr['Editor-in-chief URL'].value_counts()

                           23653
/wiki/Dan_DiDio             5800
/wiki/Dick_Giordano         5626
/wiki/Bobbie_Chase          5478
/wiki/Whitney_Ellsworth     4385
                           ...  
/wiki/Otto_Binder              1
/wiki/Phyllis_Reed             1
/wiki/Scott_Nybakken           1
/wiki/Will_Eisner              1
/wiki/John_Layman              1
Name: Editor-in-chief URL, Length: 81, dtype: int64

In [317]:
new_columns = ['URL', 'Gender', 'Birth', 'Death']
gender_ed_wr = pd.DataFrame(columns=new_columns)

for URL in dc_ed_wr['Editor-in-chief URL'].unique():
    if (URL is not '') and ('/wiki/' in URL):
        if URL[0] is not '/':
            URL = URL[1:]
            
        # Initialize variables
        gender, birth, death = '', '', ''

        # Get URL and use html parser
        URL = 'https://dc.fandom.com' + URL

        r = requests.get(URL)
        page_body = r.text

        soup = BeautifulSoup(page_body, 'html.parser')

        URL = URL.replace('https://dc.fandom.com','')

        # Parsing
        side_tab = soup.find_all('aside', class_='portable-infobox pi-background pi-theme-wikia pi-layout-default')

        for p in side_tab:
            for pp in p.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
                if pp and pp.find_next('h3', class_="pi-data-label pi-secondary-font"):
                    if pp.find_next('h3', class_="pi-data-label pi-secondary-font").text == 'Gender':
                        if pp.find('div', class_='pi-data-value pi-font'):
                            gender = pp.find('div', class_='pi-data-value pi-font').text
                    if pp.find_next('h3', class_="pi-data-label pi-secondary-font").text == 'Date of Birth':
                        if pp.find('div', class_='pi-data-value pi-font'):
                            birth = pp.find('div', class_='pi-data-value pi-font').text
                    if pp.find_next('h3', class_="pi-data-label pi-secondary-font").text == 'Date of Death':
                        if pp.find('div', class_='pi-data-value pi-font'):
                            death = pp.find('div', class_='pi-data-value pi-font').text

        if URL not in gender_ed_wr['URL'].unique():
            editor = pd.DataFrame([[URL, gender, birth, death]], columns = new_columns)
            gender_ed_wr = gender_ed_wr.append(editor)

gender_ed_wr.head()

,URL,Gender,Birth,Death
0,/wiki/Karen_Berger,Female,"February 26, 1958",
0,/wiki/Jenette_Kahn,Female,1948,
0,/wiki/Joe_Orlando,Male,"April 4, 1927","December 23, 1998"
0,/wiki/Shelly_Bond,Female,,
0,/wiki/Dick_Giordano,Male,"July 20, 1932","March 27, 2010"


In [318]:
gender_ed_wr

,URL,Gender,Birth,Death
0,/wiki/Karen_Berger,Female,"February 26, 1958",
0,/wiki/Jenette_Kahn,Female,1948,
0,/wiki/Joe_Orlando,Male,"April 4, 1927","December 23, 1998"
0,/wiki/Shelly_Bond,Female,,
0,/wiki/Dick_Giordano,Male,"July 20, 1932","March 27, 2010"
...,...,...,...,...
0,/wiki/Mike_W._Barr,Male,"May 30, 1952",
0,/wiki/Deborah_Marvin,Female,,
0,/wiki/John_Layman,Male,"August 2, 1969",
0,/wiki/Irwin_Donenfeld,Male,"March 1, 1926","November 29, 2004"


In [319]:
save_to_pickle(gender_ed_wr,'dc_ed_attributes')

In [14]:
with open('dc_wr_attributes.txt', 'rb') as f:
    dc_wr = pickle.load(f)

In [15]:
with open('dc_ed_attributes.txt', 'rb') as f:
    dc_ed = pickle.load(f)

In [19]:
dc_ed_wr = dc_ed.append(dc_wr)

In [20]:
dc_ed_wr

,URL,Gender,Birth,Death
0,/wiki/Karen_Berger,Female,"February 26, 1958",
0,/wiki/Jenette_Kahn,Female,1948,
0,/wiki/Joe_Orlando,Male,"April 4, 1927","December 23, 1998"
0,/wiki/Shelly_Bond,Female,,
0,/wiki/Dick_Giordano,Male,"July 20, 1932","March 27, 2010"
...,...,...,...,...
0,/wiki/Bill_Kunkel,Male,,
0,/wiki/Carl_Pfeufer,Male,,
0,/wiki/Marc_Swayze,Male,,
0,/wiki/David_Anthony_Kraft,Male,,


In [21]:
save_to_pickle(dc_ed_wr,'dc_ed_wr_attributes')

# Analysis

### Marvel

In [21]:
with open('../marvel_ed_attributes.txt', 'rb') as f:
    marvel_ed = pickle.load(f)
with open('../marvel_wr_attributes.txt', 'rb') as f:
    marvel_wr = pickle.load(f)

In [22]:
marvel_ed['Gender'].value_counts()

Male      55
Female     9
           2
male       1
Name: Gender, dtype: int64

In [23]:
marvel_wr['Gender'].value_counts()

Male          1319
Female         167
                72
Non-Binary       2
Other            1
None             1
Name: Gender, dtype: int64

In [24]:
marvel_ed.loc[marvel_ed['Gender']=='male','Gender']='Male'
marvel_ed.loc[marvel_ed['Gender']=='','Gender']='Unknown'

marvel_wr.loc[marvel_wr['Gender']=='','Gender']='Unknown'

**The gender `None` and `Other` corresponds to a "Studio", i.e. the writer is a group of people so there is no specified gender. We denote them as `Unknown`.**

In [25]:
marvel_wr[marvel_wr['Gender']=='Other']

,URL,Gender,Birth,Death
0,/wiki/Man_of_Action,Other,,


In [26]:
marvel_ed.loc[marvel_ed['Gender']=='None','Gender']='Unknown'
marvel_ed.loc[marvel_ed['Gender']=='Other','Gender']='Unknown'

marvel_wr.loc[marvel_wr['Gender']=='None','Gender']='Unknown'
marvel_wr.loc[marvel_wr['Gender']=='Other','Gender']='Unknown'

In [27]:
marvel_ed['Gender'].value_counts()

Male       56
Female      9
Unknown     2
Name: Gender, dtype: int64

In [29]:
marvel_wr['Gender'].value_counts()

Male          1319
Female         167
Unknown         74
Non-Binary       2
Name: Gender, dtype: int64

In [30]:
save_to_pickle(marvel_ed,'marvel_ed_attributes')
save_to_pickle(marvel_wr,'marvel_wr_attributes')

### DC

In [31]:
with open('../dc_ed_attributes.txt', 'rb') as f:
    dc_ed = pickle.load(f)
with open('../dc_wr_attributes.txt', 'rb') as f:
    dc_wr = pickle.load(f)

In [32]:
dc_ed['Gender'].value_counts()

Male      66
Female    11
           3
Name: Gender, dtype: int64

In [33]:
dc_wr['Gender'].value_counts()

Male      416
Female     39
           33
Name: Gender, dtype: int64

In [34]:
dc_ed.loc[dc_ed['Gender']=='','Gender']='Unknown'
dc_wr.loc[dc_wr['Gender']=='','Gender']='Unknown'

In [35]:
dc_ed['Gender'].value_counts()

Male       66
Female     11
Unknown     3
Name: Gender, dtype: int64

In [36]:
dc_wr['Gender'].value_counts()

Male       416
Female      39
Unknown     33
Name: Gender, dtype: int64

In [37]:
save_to_pickle(dc_ed,'dc_ed_attributes')
save_to_pickle(dc_wr,'dc_wr_attributes')